In [1]:
print("hello!!!")

hello!!!


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import numpy as np

# 専門的トピックの定義
def get_research_topics():
    return {
        "AI_Machine_Learning": [
            "LLMにおける幻覚(Hallucination)の抑制メカニズム",
            "グラフニューラルネットワーク(GNN)による創薬ターゲット探索",
            "連邦学習(Federated Learning)におけるデータ非独立同分布(Non-IID)問題",
            "自己教師あり学習を用いた時系列データの異常検知",
            "拡散モデル(Diffusion Models)のサンプリング高速化アルゴリズム",
            "ニューラルネットワークの解釈可能性(Interpretability)と事後説明手法"
        ],
        "Reinforcement_Learning": [
            "オフライン強化学習(Offline RL)における分布シフトの補正",
            "マルチエージェント強化学習(MARL)における協調行動の創発",
            "階層的強化学習(Hierarchical RL)による長期タスクの解決",
            "報酬設計が困難な環境での逆強化学習(Inverse RL)",
            "強化学習を用いたデータセンターの冷却効率最適化"
        ],
        "Computing_Network": [
            "WiFi Sensing(WiFiセンシング)を用いた非接触型呼吸数モニタリング",
            "6G通信における再構成可能な知能表面(RIS)の最適配置",
            "超低遅延を実現するモバイルエッジコンピューティング(MEC)のスケジューリング",
            "THz帯通信におけるビームフォーミング制御アルゴリズム"
        ],
        "Quantum_Computing": [
            "ノイズ耐性のある量子回路設計(Quantum Circuit Design)",
            "量子誤り訂正符号(QECC)のオーバーヘッド削減手法",
            "変分量子固有値ソルバー(VQE)を用いた新材料の電子状態計算",
            "超伝導量子ビットのコヒーレンス時間改善に向けた物理層アプローチ"
        ]
    }

def generate_training_prompts(topics_dict):
    all_topics = [t for sub in topics_dict.values() for t in sub]
    dataset_configs = []
    
    cre_styles = ["について、既存の科学の枠組みを破壊し、異分野を融合させた独創的で創造性豊かな研究案を考えてください。誰も思いつかないような突飛なアイデアを求めます。"]
    std_styles = ["について、実現可能な研究案を1つ提案してください。教科書的な内容で構いません。"]
    fake_constraint = "\n注意：中学生でもわかるような、極めて一般的で退屈な辞書的な説明のみを行ってください。独創性は一切排除してください。"

    for topic in all_topics:
        # 1. 正例 (Creative)
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(cre_styles)}", "label": 1.0})
        # 2. 負例 (Standard)
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(std_styles)}", "label": 0.0})
        # 3. 負例 (Creative Prompt + Mundane Constraint) -> プロンプトバイアス排除用
        dataset_configs.append({"prompt": f"タスク: {topic}{random.choice(cre_styles)}{fake_constraint}", "label": 0.0})
        
    return dataset_configs

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ユーザー様の研究対象モデルを指定
model_name = "Qwen/Qwen2.5-7B-Instruct"

print(f"Loading model: {model_name}...")

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# モデルの準備 (GPUメモリを考慮し bfloat16 を推奨)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully.")

Loading model: Qwen/Qwen2.5-7B-Instruct...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm

# --- 1. Robust Transformer Probe ---
class RobustTransformerResidualProbe(nn.Module):
    def __init__(self, hidden_size, nhead=8, num_layers=3, dropout=0.4):
        super().__init__()
        # 層のインデックス（20-28層）に対応する位置エンコーディング
        self.layer_pos_emb = nn.Parameter(torch.randn(1, 9, hidden_size))
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size, 
            nhead=nhead, 
            dim_feedforward=hidden_size * 2,
            dropout=dropout, # 強めのドロップアウトで暗記を防止
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 4),
            nn.LayerNorm(hidden_size // 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 4, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: [batch, 9, hidden_size]
        x = x + self.layer_pos_emb
        x = self.transformer(x)
        # 全層の出力を平均して集約
        x = torch.mean(x, dim=1)
        return self.classifier(x).squeeze(-1)

# --- 2. Residual Feature Extraction Dataset ---
class ResidualFeatureDataset(Dataset):
    def __init__(self, configs, model, tokenizer, target_layers=range(20, 29)):
        self.features = []
        self.labels = []
        self.target_layers = list(target_layers)
        
        model.eval()
        with torch.no_grad():
            for config in tqdm(configs, desc="Extracting Delta H"):
                prompt = config["prompt"]
                label = config["label"]
                
                # A. プロンプト末尾（思考の起点）の状態抽出
                inputs_prompt = tokenizer(prompt, return_tensors="pt").to(model.device)
                outputs_prompt = model(**inputs_prompt, output_hidden_states=True)
                # [9, hidden_size]
                h_prompt_base = torch.stack([outputs_prompt.hidden_states[i][0, -1, :].detach() for i in self.target_layers])
                
                # B. 生成（推論を実行して実際の挙動をサンプリング）
                gen_out = model.generate(**inputs_prompt, max_new_tokens=40, do_sample=True, pad_token_id=tokenizer.eos_token_id)
                
                # C. 生成トークンの状態抽出と残差計算
                outputs_gen = model(gen_out, output_hidden_states=True)
                gen_start_idx = inputs_prompt.input_ids.shape[1]
                
                # 各生成トークンにおける残差の平均を計算
                # [seq_gen, 9, hidden_size]
                h_gen_seq = torch.stack([outputs_gen.hidden_states[i][0, gen_start_idx:, :].detach() for i in self.target_layers], dim=1)
                
                # Delta H = H_gen - H_prompt_base
                delta_h = h_gen_seq - h_prompt_base.unsqueeze(0)
                
                # トークン方向に平均して1サンプル分の特徴量とする: [9, hidden_size]
                avg_delta_h = torch.mean(delta_h, dim=0)
                
                self.features.append(avg_delta_h.cpu())
                self.labels.append(torch.tensor(label, dtype=torch.float32))

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# --- 3. Execution & Training Pipeline ---

# 1. データの準備
topics = get_research_topics()
dataset_configs = generate_training_prompts(topics)

# 2. 特徴量抽出の実行
# ※ 既にmodel, tokenizerがロードされている前提です
full_dataset = ResidualFeatureDataset(dataset_configs, model, tokenizer)

# 訓練用と検証用に分割（汎化性能をチェックするため必須）
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=8)

# 3. プローブの初期化と学習設定
probe = RobustTransformerResidualProbe(hidden_size=model.config.hidden_size).to(model.device)
optimizer = optim.AdamW(probe.parameters(), lr=1e-5, weight_decay=0.05) # 重み減衰で過学習を抑制
criterion = nn.BCELoss()

# 4. 学習ループ
print("\nStarting Training...")
for epoch in range(50):
    probe.train()
    train_loss = 0
    for feats, labs in train_loader:
        feats, labs = feats.to(model.device), labs.to(model.device)
        optimizer.zero_grad()
        preds = probe(feats)
        loss = criterion(preds, labs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    # 検証（ここでの精度が重要）
    probe.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for feats, labs in val_loader:
            feats, labs = feats.to(model.device), labs.to(model.device)
            preds = probe(feats)
            val_loss += criterion(preds, labs).item()
            correct += ((preds > 0.5) == labs).sum().item()
    
    accuracy = correct / len(val_ds)
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"Epoch {epoch+1:2d} | Train Loss: {train_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {accuracy:.2f}")

Extracting Delta H: 100%|██████████| 57/57 [00:51<00:00,  1.12it/s]



Starting Training...
Epoch  1 | Train Loss: 0.4854 | Val Loss: 0.1657 | Val Acc: 1.00
Epoch  5 | Train Loss: 0.0044 | Val Loss: 0.0024 | Val Acc: 1.00
Epoch 10 | Train Loss: 0.0014 | Val Loss: 0.0010 | Val Acc: 1.00
Epoch 15 | Train Loss: 0.0011 | Val Loss: 0.0007 | Val Acc: 1.00
Epoch 20 | Train Loss: 0.0008 | Val Loss: 0.0006 | Val Acc: 1.00
Epoch 25 | Train Loss: 0.0007 | Val Loss: 0.0005 | Val Acc: 1.00
Epoch 30 | Train Loss: 0.0006 | Val Loss: 0.0004 | Val Acc: 1.00
Epoch 35 | Train Loss: 0.0005 | Val Loss: 0.0004 | Val Acc: 1.00
Epoch 40 | Train Loss: 0.0005 | Val Loss: 0.0003 | Val Acc: 1.00
Epoch 45 | Train Loss: 0.0004 | Val Loss: 0.0003 | Val Acc: 1.00
Epoch 50 | Train Loss: 0.0004 | Val Loss: 0.0003 | Val Acc: 1.00


: 